## Convolutional neural network 
- >Image classification
    -  split traindata and testdata
    -  transform PIL to tensor [255] -> [0-1]
    -  normalize tensor [0-1] -> [-1-1]
    -  put train/test data to Dataloader(shuffle,batch_size,num_workers) respectively


### Import libraries

In [1]:
import torch 
import torch.nn as nn                 # Activation Funciton , Algo
from torch.autograd import Variable   
import torch.utils.data as Data       # Data loader 
import torchvision                    # Download dataset
import matplotlib.pyplot as plt 

### Main program

In [2]:
# Hyper parameters
EPOCH = 1
LR = 0.001
BATCH_SIZE = 50
DOWNLOAD_CIFAR10 = False


train_dataset = torchvision.datasets.CIFAR10(
    root = "./CIFAR10",
    train = True,
    # torchvision ToTensor() 把 pixel to tensor [255] 轉換成 [0 - 1] 區間 
    # torchvision.transforms.Normalize (mean , std) 把 tensor [0-1] 轉換成 [-1 - 1] 區間做標準化 使模型更快收斂
    transform = torchvision.transforms.Compose([torchvision.transforms.Resize([28,28]),torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),  # RGB(pixel) [0-255] -> Tensor [0-1]
    download = DOWNLOAD_CIFAR10,
    )
test_dataset = torchvision.datasets.CIFAR10(
    root = "./CIFAR10",
    train = False,
    transform = torchvision.transforms.Compose([torchvision.transforms.Resize([28,28]),torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    download = DOWNLOAD_CIFAR10,
)

train_dataloader = Data.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True , 
    num_workers = 2 
)

test_dataloader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle = True
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



### CNN 
- >conv1 (Conv2d , ReLU , Maxpool)    [n , 3 , 28 ,28]
- >conv2 (Conv2d , ReLU , Maxpool)    [n , 3 , 28 ,28]
- >output 
    

In [35]:
class CNN(nn.Module):
    def __init__(self,num_classes:int) -> None:
        super(CNN,self).__init__()
        self.num_classes = num_classes
        # 第一層卷積層 aka 過濾器
        self.conv1 = nn.Sequential(
            # 維度 [ BATCH , 3高(RGB) , 28長 , 28寬]
            nn.Conv2d( 
                in_channels = 3,      # RGB = 3 , 黑白圖片（灰） = 1                   , 圖片進來的高度
                out_channels = 16,    # 在同一個過濾器上有幾個再細分成16個 提取16個特徵     , 圖片輸出的高度
                kernel_size = 5,      # 掃描區域 5 個像素點 
                stride = 1,           # 掃描一次窗口移動幾個 pixel  
                padding = 2,          # 圍裙 在圖片多出來的地方可作特徵收集   , padding = (kernal_size -1) / 2 = (5-1)/2
            ),
            # 維度 [ BATCH , 16高(RGB) , 28長 , 28寬]                     
            nn.ReLU(),
            # 維度 [ BATCH , 16高(RGB) , 14長 , 14寬]
            nn.MaxPool2d(2)         # 選擇特徵 ex: 最大值 ,以 2x2 kernal size 篩選特徵
        )   
            # 維度 [ BATCH , 16高 , 14長 , 14寬]
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,5,1,2),   # 維度 [ BATCH , 32高(RGB) , 14 , 14]
            nn.ReLU(),                # 維度 [ BATCH , 32高(RGB) , 14 , 14]
            nn.MaxPool2d(2)           # 維度 [ BATCH , 32高(RGB) , 7 , 7]
        )
        self.out = nn.Linear(32 * 7 * 7 , 10)   #(維度,label數量) == (input , output)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)             # (batch - > 32 ,7 ,7)
        # 轉換成 2d array 
        x = x.view(x.size(0), -1)     # 數據展平 ( batch , 32 * 7 * 7)   , x.size(0) == batch_size
        # 數據展平之後 -> 2d array 才能進入 ouput layer
        output = self.out(x)

        return output

cnn = CNN(len(classes))
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


### Training

In [36]:
optimizer = torch.optim.Adam(cnn.parameters() , lr=LR)
loss_func = nn.CrossEntropyLoss() # for label model 


# Training and testing 
for epoch in range(EPOCH):
    for step , (x,y) in enumerate(train_dataloader):
        b_x = Variable(x)
        b_y = Variable(y)

        output = cnn(b_x)
        loss = loss_func(output ,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
             print('Epoch: {} | Step: {} | Loss: {}'.format(epoch + 1, step, loss))

Epoch: 1 | Step: 0 | Loss: 2.298053503036499
Epoch: 1 | Step: 100 | Loss: 1.7587240934371948
Epoch: 1 | Step: 200 | Loss: 1.36772882938385
Epoch: 1 | Step: 300 | Loss: 1.5020583868026733
Epoch: 1 | Step: 400 | Loss: 1.4992878437042236
Epoch: 1 | Step: 500 | Loss: 1.5369389057159424
Epoch: 1 | Step: 600 | Loss: 1.3677856922149658
Epoch: 1 | Step: 700 | Loss: 1.3148845434188843
Epoch: 1 | Step: 800 | Loss: 1.1728345155715942
Epoch: 1 | Step: 900 | Loss: 1.2440508604049683
